In [ ]:
!apt-get -qq install awscli
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab07052019_PodCastRound2.tar.gz ./ 
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab09272017_PodCastRound3.tar.gz ./
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/test/OrcasoundLab09272017_Test.tar.gz ./
!tar -xzf OrcasoundLab09272017_PodCastRound3.tar.gz
!tar -xzf OrcasoundLab07052019_PodCastRound2.tar.gz
!tar -xzf OrcasoundLab09272017_Test.tar.gz
!pip -q install ketos==2.0.0b4
!pip -q install pysoundfile
!pip install pydub


Selecting previously unselected package sgml-base.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../00-sgml-base_1.29_all.deb ...
Unpacking sgml-base (1.29) ...
Selecting previously unselected package python3-yaml.
Preparing to unpack .../01-python3-yaml_3.12-1build2_amd64.deb ...
Unpacking python3-yaml (3.12-1build2) ...
Selecting previously unselected package python3-six.
Preparing to unpack .../02-python3-six_1.11.0-2_all.deb ...
Unpacking python3-six (1.11.0-2) ...
Selecting previously unselected package python3-dateutil.
Preparing to unpack .../03-python3-dateutil_2.6.1-1_all.deb ...
Unpacking python3-dateutil (2.6.1-1) ...
Selecting previously unselected package xml-core.
Preparing to unpack .../04-xml-core_0.18_all.deb ...
Unpacking xml-core (0.18) ...
Selecting previously unselected package docutils-common.
Preparing to unpack .../05-docutils-common_0.14+dfsg-3_all.deb ...
Unpacking docutils-common (0.14+dfsg-3) ...
Selecting prev

In [ ]:


import numpy as np
from os import listdir
from os.path import isfile, join
from scipy import signal
import soundfile as sf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker



In [ ]:
from ketos.data_handling import selection_table


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from pydub import AudioSegment
from os import listdir
from os.path import isfile, join
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from ketos.data_handling import selection_table as sl
import librosa


# Function to add the end time
def add_end(filename):
    """
    Adds the parameter called end which is the
    end_time to the .tsv/csv file.

    Returns:same tsv/csv file with an extra parameter of end_time.
    """
    filename["end"] = filename["start"] + filename["duration_s"]


# Generate mean and annot-train
def duration_mean(filename):
    """
    Reads the .tsv/csv file and calculates the mean.

    Returns:mean and the pandas tsv/csv file.
    """
    annot_train = pd.read_csv(filename, sep='\t')
    mean = annot_train['duration_s'].mean()
    return annot_train, mean


# Function to find extract filename and start time
def fname_stime(filename):
    """
    Reads the file name and the start time from the given file from
    which the calls are to be extracted.

    Returns:file_name and the start_time.
    """
    file_name = filename.iloc[:, 0].values
    start_time = filename.iloc[:, 1].values
    return file_name, start_time


# Function to extract audio from the .wav files to generate
# complete positive and negative calls
def extract_audio(
      label, filename, output_directory, position,
      file_location, call_time_in_seconds):
    """
    This function extracts the audio from the .wav files to generate
    smaller audio clips of particular time frame which is used to
    get the complete positive and negative calls.

    Here the enter the call duration in variable call_time_in_seconds
    Since the audio segment assumes 3000 as 3 second, therefore, we
    would multiply the time entered by user by 1000.

    Enter the output directory name in the output_directory
    Returns:the extracted calls of shorter time frames containing only
    the calls or no calls.
    """

    # make directory if it does not exist
    if not os.path.isdir(output_directory):
        os.mkdir(output_directory)
    outpath = os.path.join(base_path,output_directory)
    file_name = filename.iloc[:, 0].values
    start_time = filename.iloc[:, position].values
    i = 0
    o = 0
    call_time_in_seconds = call_time_in_seconds*1000
    os.chdir(file_location)
    # Enter the file location of the calls directory
    for x in file_name:
        AUDIO_FILE = x
        sound = AudioSegment.from_file(AUDIO_FILE)
        p = start_time[i]
        p = p * 1000
        print(p)
        i = i + 1
        o = p + call_time_in_seconds
        call = sound[p:o]
        call.export(outpath + label + "calls{0}.wav".format(i),
                    format="wav")




# Enter the base_path, the current path, and the name of the tsv file that
# you want to read
base_path = os.getcwd()
path = ''
audio_file_path = os.path.join(base_path, 'Round2_OS_07_05/wav')
audio_file_path2 = os.path.join(base_path, 'Round3_OS_09_27_2017/wav')

# Enter the path of the standardized tsv's
annot_train2, mean2 = duration_mean("/content/podcast2.tsv")
annot_train3, mean3 = duration_mean("/content/podcast3.tsv")
annot_test, mean_test = duration_mean("/content/v10_test.tsv")

add_end(annot_train2)
add_end(annot_train3)
add_end(annot_test)
annot_train2.head()


# Enter the time_duration in seconds of the call_time that you want to extract
# from the audio file
call_time = 3


# Extract the audio of the calls
extract_audio(
    "rnd2_cl", annot_train2,
    "pod_calls/", 1,
    audio_file_path, call_time)

extract_audio(
    'round3_calls', annot_train3,
    "pod_calls/", 1,
    audio_file_path2, call_time)


'''
Since we also want the negative audio, we would generate
negative sound by extracting it from the background sound
what are not present in  the call duration. For that first
we create a tsv that generates time-interval from the tsv
files that are not within the start-time and end-time
'''




# Saving these tsv files for future use



49765.625
41046.0069444444
37345.4861111111
42917.534722222204
45980.034722222204
52700.5208333333
55295.1388888889
1147.64052741152
26115.197779319897
29995.0728660652
34725.0520471894
52485.426786953496
36554.4760582929
13883.906030855502
17708.3800841515
21964.095371669
19672.159887798
27846.966527196604
29329.9511854951
34229.951185495105
56773.709902370996
37544.4560669456
46172.41980474201
6233.23983169705
42204.9438990182
56264.09537166899
58972.44491458839
13916.549789621302
7261.1576011157595
8286.26220362622
10550.0348675035
11318.8633193863
34554.5676429568
52237.62203626221
53647.1408647141
14451.171875
47755.859375
56895.5078125
31508.298465829805
37607.6708507671
1250.0
58064.3398354815
13398.4375
27945.3125
17035.15625
38233.3984375
46798.828125
57363.1450488145
11881.54296875
20336.9140625
34300.0
41412.04351204351
46430.0699300699
51488.28125
53785.15625
57134.765625
129.12860154603
5552.52986647927
8522.487702037952
21091.0049191848
24534.434293745606
30216.0927617709

'\nSince we also want the negative audio, we would generate\nnegative sound by extracting it from the background sound\nwhat are not present in  the call duration. For that first\nwe create a tsv that generates time-interval from the tsv\nfiles that are not within the start-time and end-time\n'

In [ ]:
extract_audio(
    'test', annot_test,
    "test_calls/", 1,
    "/content/OrcasoundLab09272017_Test/wav/", call_time)

11643.564359999998
15594.05941
53900.0
59781.486399999994
6592.881944
23011.28472
29519.09722
50769.44444
20709.25197
22725.19685
41650.0
43280.118109999996
52125.19685
54671.65354
56687.59843
60295.078740000004
0.0
2666.724257
4688.355218
7815.618521
16091.77609
18457.118179999998
20991.77609
23187.802349999998
28045.473390000003
30011.23013
31704.38839
47311.921220000004
51957.94748
60069.86869
4432.118056
7945.486111
10506.07639
12760.41667
16299.30556
17685.9375
20331.59722
23351.5625
33253.64583
34538.19444
38221.70139
46550.0
49042.534719999996
54027.60417
578.472222
5282.8125
12888.02083
19217.1875
25010.416670000002
34300.0
36750.0
49000.0
1745.84323
6401.4251779999995
9800.0
18727.07838
22971.41726
28307.878070000002
29496.99129
32577.434680000002
35595.80364
49000.0
51207.52177
52930.08709
935.7638890000001
3360.2430560000003
5486.979167
7350.0
10165.79861
12250.0
14631.94444
16767.1875
18800.34722
20842.013890000002
22245.65972
24500.0
25988.71528
27052.083329999998
29697.74

In [ ]:
ne=generate_negative_tsv(annot_test, 3, "/content/OrcasoundLab09272017_Test/wav/")

In [ ]:
ne.tail()

,filename,sel_id,start,end,label
96,OS_9_27_2017_11_58_00__0003.wav,1,6.006218,9.006218,0
97,OS_9_27_2017_11_58_00__0003.wav,2,7.652121,10.652121,0
98,OS_9_27_2017_11_58_00__0003.wav,3,12.472731,15.472731,0
99,OS_9_27_2017_11_58_00__0003.wav,4,14.043252,17.043252,0
100,OS_9_27_2017_11_58_00__0003.wav,5,30.754387,33.754387,0


In [ ]:
extract_audio(
    'test_neg', ne,
    "/content/OrcasoundLab09272017_Test/wav/test_neg_calls/", 2,
    "/content/OrcasoundLab09272017_Test/wav/", call_time)

20240.113642289045
22639.713101894376
33629.73272162322
34923.31555753733
46850.94444225992
1145.6498022619003
29199.011285827026
30656.514473180294
32800.50013039977
32840.17999596242
45630.83501420647
48962.15968223183
11443.863058053865
29378.173163505893
27917.05582362937
35837.93466220066
42668.07345107583
47400.46394027706
57092.70435133351
11277.349969412853
28323.475935921124
28929.285510368005
43448.27447897165
45328.726946386836
52703.70344443603
55739.54819646844
40587.109087253906
42689.297129571685
56275.23976778724
11300.954804211415
36939.17455064172
53769.91464906144
21497.167856643613
30637.727432258318
44140.24494900502
5993.528459676213
10138.943087927488
16172.14619878689
25394.611417823286
26945.7585752682
29719.08650526598
53024.10043287762
1107.8066677158631
5433.396841351168
14513.831169744095
17815.817555161517
34939.17764248158
39618.78702495403
46213.19909976091
46759.31075380322
14421.315590679911
37789.46001839949
38113.4249019467
536.5027083561245
20183.31

In [ ]:

def generate_negative_tsv(
        call_annotations,
        call_time,
        files_dir
):
    """Generates .tsv file containing start-time and end-time of
    the negative calls.

    Since we also want the pure negative samples, that do not contain
    the calls we would generate a .tsv file which contains the interval
    not in the start-time and duration of the .tsv containing the calls.
    And since any area that does not contain the calls would contain no
    call or the background noise, we would use this start-time and
    duration to extract audio from the audio files.

    Args:
        call_annotations: The .tsv file containing the calls.
        call_time: The duration for which you want to generate negative
                    calls.
        files_dir: The directory that contains the audio data.

    Returns:
        A pandas dataframe containing start_time and end_time of the
        background sounds.
    """
    standardized_annotations = selection_table.standardize(
        table=call_annotations,
        signal_labels=["SRKWs"],
        mapper={"wav_filename": "filename"},
        trim_table=True
    )

    positives_call_duration = selection_table.select(
        annotations=standardized_annotations,
        length=call_time
    )
    file_durations = selection_table.file_duration_table(
        files_dir
    )

    # Generate a .tsv file which does not include any calls.
    negatives_annotations = selection_table.create_rndm_backgr_selections(
        annotations=standardized_annotations,
        files=file_durations,
        length=call_time,
        num=len(positives_call_duration),
        trim_table=True
    )

    negative_tsv_generated = negatives_annotations.reset_index(level=[0, 1])
    return negative_tsv_generated



In [ ]:

def generate_negative_tsv(
        call_annotations,
        call_time,
        files_dir
):
    """Generates .tsv file containing start-time and end-time of
    the negative calls.

    Since we also want the pure negative samples, that do not contain
    the calls we would generate a .tsv file which contains the interval
    not in the start-time and duration of the .tsv containing the calls.
    And since any area that does not contain the calls would contain no
    call or the background noise, we would use this start-time and
    duration to extract audio from the audio files.

    Args:
        call_annotations: The .tsv file containing the calls.
        call_time: The duration for which you want to generate negative
                    calls.
        files_dir: The directory that contains the audio data.

    Returns:
        A pandas dataframe containing start_time and end_time of the
        background sounds.
    """
    standardized_annotations = selection_table.standardize(
        table=call_annotations,
        signal_labels=["SRKWs"],
        mapper={"wav_filename": "filename"},
        trim_table=True
    )

    positives_call_duration = selection_table.select(
        annotations=standardized_annotations,
        length=call_time
    )
    file_durations = selection_table.file_duration_table(
        files_dir
    )

    # Generate a .tsv file which does not include any calls.
    negatives_annotations = selection_table.create_rndm_backgr_selections(
        annotations=standardized_annotations,
        files=file_durations,
        length=call_time,
        num=len(positives_call_duration),
        trim_table=True
    )

    negative_tsv_generated = negatives_annotations.reset_index(level=[0, 1])
    return negative_tsv_generated



In [ ]:
negative_generated_tsv22

,filename,sel_id,start,end,label
0,1562337136_0004.wav,0,2.977141,5.977141,0
1,1562337136_0004.wav,1,4.638070,7.638070,0
2,1562337136_0004.wav,2,13.625698,16.625698,0
3,1562337136_0004.wav,3,16.905728,19.905728,0
4,1562337136_0004.wav,4,17.428979,20.428979,0
...,...,...,...,...,...
352,1562344334_001c.wav,0,7.825907,10.825907,0
353,1562344334_001c.wav,1,26.908543,29.908543,0
354,1562344334_001c.wav,2,35.576592,38.576592,0
355,1562344334_001c.wav,3,40.701359,43.701359,0


In [ ]:
call_annotations = pd.read_csv("/content/podcast2.tsv", sep="\t")
call_length_mean = call_annotations["duration_s"].mean()
#rint("The mean of the call duration is {}".format(call_length_mean))
call_annotations["end"] = call_annotations["start"] + call_annotations["duration_s"]

# extract the audio of the calls

# generate negative .tsv file
negative_generated_tsv = generate_negative_tsv(
                                    call_annotations,
                                    call_time, "/content/Round2_OS_07_05/wav")
extract_audio(  
        "rc2cal",negative_generated_tsv22,
        "/content/negtive_old/",2
        "/content/Round2_OS_07_05/wav/"
    )


SyntaxError: ignored

In [ ]:
negative_generated_tsv22.tail()

,filename,sel_id,start,end,label
352,1562344334_001c.wav,0,7.825907,10.825907,0
353,1562344334_001c.wav,1,26.908543,29.908543,0
354,1562344334_001c.wav,2,35.576592,38.576592,0
355,1562344334_001c.wav,3,40.701359,43.701359,0
356,1562344334_001c.wav,4,45.333012,48.333012,0


In [ ]:
extract_audio(  
        "rc2cal",negative_generated_tsv22,
        "/content/negtive_old/",2,
        "/content/Round2_OS_07_05/wav/",3
    )

2977.140664329337
4638.070441142397
13625.69771073913
16905.728498286193
17428.978770150614
17635.039952034254
19362.21773591246
19967.10319992961
21421.884215877766
30344.004519346752
2412.7212848191275
3334.834347101989
20898.242332319172
22413.574356945603
23387.851657489777
36891.22494390155
38379.69646061873
44153.70891015877
45822.1334536004
50967.250589218624
56860.71788862803
4376.309571237655
10573.714855063656
11083.455216068302
46458.56685944344
55825.72466970666
395.2749841671732
26784.632994130847
29483.248395160073
33926.26526933162
45068.38482570194
48823.47417788438
52338.87840592166
57105.81348048717
1991.1401917560454
2985.7286376508796
5977.178613083253
8989.986031777506
18396.035915054283
40708.1795028717
41407.67505835504
48304.59594014923
16422.173648800595
18478.67424678452
22457.560277806693
24127.001160774
28181.936209985222
30711.587769940423
34020.62394396012
36448.67215892282
45340.467649299004
2811.487871023303
9984.244252520966
10465.114027682148
14700.826

In [ ]:
call_annotations22 = pd.read_csv("/content/podcast2.tsv", sep="\t")
#rint("The mean of the call duration is {}".format(call_length_mean))
call_annotations22["end"] = call_annotations22["start"] + call_annotations22["duration_s"]

# extract the audio of the calls

# generate negative .tsv file
negative_generated_tsv22 = generate_negative_tsv(
                                    call_annotations22,
                                    call_time, "/content/Round2_OS_07_05/wav")
extract_audio("roun2ca", "call",
        "/content/negtive_old/",2,
        "/content/Round2_OS_07_05/wav/",
        3
    )


TypeError: ignored

In [ ]:
call_annotations33 = pd.read_csv("/content/podcast3.tsv", sep="\t")
#rint("The mean of the call duration is {}".format(call_length_mean))
call_annotations33["end"] = call_annotations33["start"] + call_annotations33["duration_s"]

# extract the audio of the calls

# generate negative .tsv file
negative_generated_tsv33 = generate_negative_tsv(
                                    call_annotations33,
                                    call_time, "/content/Round3_OS_09_27_2017/wav/")
extract_audio("ro33ca", negative_generated_tsv33,
        "/content/negtive_old/",2,
       "/content/Round3_OS_09_27_2017/wav/",
        3
    )

8485.366623111337
15812.55461832189
20956.93790296738
21954.50644313863
23139.950010335393
25440.053510058355
30652.446083086015
40624.54975748645
54715.669469913126
354.10720097841875
12348.290671396142
35446.97090724462
49573.07649733772
22658.572726316776
32197.083174331055
32734.342808954723
40626.63911570755
2278.169828398205
9737.913299514503
16709.132110639614
34990.135063330315
37443.2428063003
40344.84298098414
43844.90061841907
5720.533834531863
6990.1071827268315
15873.097411853252
19934.536366586457
27464.31568098592
40482.31679574156
43396.82680346743
36931.03125849029
41467.66060146371
43035.05842127458
44443.442067228374
49190.05994481091
15691.55708518764
42805.413284015514
48909.81443317304
52923.92901780329
1051.1740405236765
2188.4825105797745
8970.870943790658
10970.794870431519
11228.771448766907
33347.03791098752
33991.85219876597
34316.77787828454
38952.384431293125
46172.4869699454
57758.2313355536
32128.757063348305
50611.73230533785
11940.27240581977
12380.728

In [ ]:
# complete positive and negative calls
def extract_audio(
      label, filename, output_directory, position,
      file_location, call_time_in_seconds)

In [ ]:
generate_negative_tsv()

In [ ]:
%cd /content/

/content


In [ ]:



extract_audio(
    'rnd2_cl', negative_generated_tsv22,
    "yo_bro_negative/", 2,
    audio_file_path, call_time)

'''
extract_audio(
    'round3_calls', negatives_train33save,
    "yo_bro_negative/", 2,
    audio_file_path2, call_time)
'''

2584.385398800237
10282.966071580911
11945.167035335924
13999.627718146807
16345.994786255147
21363.52945702795
27569.307367834357
31134.725369946605
31780.85044626224
32216.027476951454
57613.07315224992
4774.28444235661
8008.526850163491
8184.31888025863
17772.154538462502
23214.16239215927
27674.622672662
29076.157515685096
29135.669104574434
37214.15138998748
47364.76090165182
55830.12644268574
7765.674504132137
9032.924079308768
15371.007623757634
18407.524100979514
41456.679163758774
55798.98371161133
57606.985109195026
2207.537423447633
3952.8421319229783
24937.996058817986
36893.12948172037
39532.80999458931
43898.64579512368
1011.3890539621195
8945.003886278726
11984.497881476445
14052.412736686507
21425.945019983203
1963.981358715671
2750.097658323796
18943.087999904375
22515.704339087108
24385.188816289716
25317.73797213879
32499.829303875737
2360.6512350929734
3639.5615026774512
4192.404236486425
4349.8485407497465
19916.67111552363
20249.64700832686
21618.61525147265
26567

KeyboardInterrupt: ignored

In [ ]:
# Please replace this with YOUR path to orcadata
basePath = r"/content/pod_calls/"
plotPath = "/content/plot_calls/"
onlyfiles = [f for f in listdir(basePath) if isfile(join(join(basePath, f)))]

for idx, file in enumerate(onlyfiles):
    data, samplerate = sf.read(join(basePath, file))
    f, t, spec = signal.spectrogram(data, samplerate)
    filename = file.split(sep=".")[0]
    i=0
    i=i+1    
    fig, ax = plt.subplots(1, 1)
    ax.specgram(data, Fs=samplerate,cmap="gray", NFFT=1024)
    
    # Rescale y axis labels from Hz to KHz
    scale_y = 1000
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    ax.yaxis.set_major_formatter(ticks_y)

    ax.set_axis_off()

    plt.savefig(join(plotPath, 
                 filename + ".png"))
    
    plt.close(fig)

In [ ]:
# Please replace this with YOUR path to orcadata
basePath = r"/content/negtive_old/"
plotPath = "/content/plot_neg22/"
onlyfiles = [f for f in listdir(basePath) if isfile(join(join(basePath, f)))]

for idx, file in enumerate(onlyfiles):
    data, samplerate = sf.read(join(basePath, file))
    f, t, spec = signal.spectrogram(data, samplerate)
    filename = file.split(sep=".")[0]
    i=0
    i=i+1    
    fig, ax = plt.subplots(1, 1)
    ax.specgram(data, Fs=samplerate,cmap="gray", NFFT=1024)
    
    # Rescale y axis labels from Hz to KHz
    scale_y = 1000
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    ax.yaxis.set_major_formatter(ticks_y)

    fig.set_size_inches(10, 10)
    ax.set_axis_off()

    plt.savefig(join(plotPath, 
                 filename + ".png"),
      
        dpi=80,
        bbox_inches="tight",
        quality=95,
        pad_inches=0.0)
    plt.close(fig)

In [ ]:
# Please replace this with YOUR path to orcadata
basePath = r"/content/OrcasoundLab09272017_Test/wav/test_neg_calls/"
plotPath = "/content/test/test_no_calls/"
onlyfiles = [f for f in listdir(basePath) if isfile(join(join(basePath, f)))]

for idx, file in enumerate(onlyfiles):
    data, samplerate = sf.read(join(basePath, file))
    f, t, spec = signal.spectrogram(data, samplerate)
    filename = file.split(sep=".")[0]
    i=0
    i=i+1    
    fig, ax = plt.subplots(1, 1)
    ax.specgram(data, Fs=samplerate,cmap="gray", NFFT=1024)
    
    # Rescale y axis labels from Hz to KHz
    scale_y = 1000
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    ax.yaxis.set_major_formatter(ticks_y)

    ax.set_axis_off()

    plt.savefig(join(plotPath, 
                 filename + ".png"))
    
    plt.close(fig)

In [ ]:
# Please replace this with YOUR path to orcadata
basePath = r"/content/OrcasoundLab09272017_Test/wav/test_neg_calls"
plotPath = "/content/test/test_no_calls/"
onlyfiles = [f for f in listdir(basePath) if isfile(join(join(basePath, f)))]

for idx, file in enumerate(onlyfiles):
    data, samplerate = sf.read(join(basePath, file))
    f, t, spec = signal.spectrogram(data, samplerate)
    filename = file.split(sep=".")[0]
    i=0
    i=i+1    
    fig, ax = plt.subplots(1, 1)
    ax.specgram(data, Fs=samplerate,cmap="gray", NFFT=1024)
    
    # Rescale y axis labels from Hz to KHz
    scale_y = 1000
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    ax.yaxis.set_major_formatter(ticks_y)

    fig.set_size_inches(10, 10)
    ax.set_axis_off()

    plt.savefig(join(plotPath, 
                 filename + ".png"),
      
        dpi=80,
        bbox_inches="tight",
        quality=95,
        pad_inches=0.0)
    plt.close(fig)

In [ ]:
!zip -r "/content/no_calls_grayscale.zip" "/content/plot_no_calls"

  adding: content/plot_no_calls/ (stored 0%)
  adding: content/plot_no_calls/round3_callscalls604.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls790.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls204.png (deflated 0%)
  adding: content/plot_no_calls/rnd2_clcalls168.png (deflated 0%)
  adding: content/plot_no_calls/rnd2_clcalls566.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls410.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls414.png (deflated 0%)
  adding: content/plot_no_calls/rnd2_clcalls713.png (deflated 0%)
  adding: content/plot_no_calls/rnd2_clcalls398.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls168.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls435.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls554.png (deflated 0%)
  adding: content/plot_no_calls/round3_callscalls106.png (deflated 0%)
  adding: content/plot_no_calls/rnd2_clcalls506.png 

In [ ]:
!zip -r "/content/calls_grayscale.zip" "/content/plot_calls"

  adding: content/plot_calls/ (stored 0%)
  adding: content/plot_calls/round3_callscalls204.png (deflated 0%)
  adding: content/plot_calls/rnd2_clcalls168.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls410.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls414.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls168.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls106.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls275.png (deflated 0%)
  adding: content/plot_calls/rnd2_clcalls112.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls167.png (deflated 1%)
  adding: content/plot_calls/rnd2_clcalls206.png (deflated 0%)
  adding: content/plot_calls/rnd2_clcalls342.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls372.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls256.png (deflated 0%)
  adding: content/plot_calls/round3_callscalls217.png (deflated 0%)
  adding: content/plot_cal

In [ ]:
!zip -r "/content/no_calls_test_graysddcale.zip" "/content/test"

  adding: content/test/ (stored 0%)
  adding: content/test/.ipynb_checkpoints/ (stored 0%)
  adding: content/test/test_calls/ (stored 0%)
  adding: content/test/test_calls/testcalls52.png (deflated 0%)
  adding: content/test/test_calls/testcalls7.png (deflated 0%)
  adding: content/test/test_calls/testcalls21.png (deflated 0%)
  adding: content/test/test_calls/testcalls79.png (deflated 0%)
  adding: content/test/test_calls/testcalls42.png (deflated 0%)
  adding: content/test/test_calls/testcalls46.png (deflated 0%)
  adding: content/test/test_calls/testcalls39.png (deflated 0%)
  adding: content/test/test_calls/testcalls3.png (deflated 0%)
  adding: content/test/test_calls/testcalls37.png (deflated 0%)
  adding: content/test/test_calls/testcalls33.png (deflated 0%)
  adding: content/test/test_calls/testcalls96.png (deflated 0%)
  adding: content/test/test_calls/testcalls49.png (deflated 0%)
  adding: content/test/test_calls/testcalls31.png (deflated 0%)
  adding: content/test/test_call

In [ ]:
nb_train_samples = sum(len(files) for _, _, files in os.walk("/content/plot_calls"))
print(nb_train_samples)

786


In [ ]:
nb_train_samples = sum(len(files) for _, _, files in os.walk("/content/negtive_old"))
print(nb_train_samples)

357


In [ ]:
!zip -r "/content/no_calls_grayscalewwss.zip" "/content/negtive_old"

  adding: content/negtive_old/ (stored 0%)
  adding: content/negtive_old/ro33cacalls48.wav (deflated 8%)
  adding: content/negtive_old/ro33cacalls159.wav (deflated 5%)
  adding: content/negtive_old/ro33cacalls137.wav (deflated 7%)
  adding: content/negtive_old/ro33cacalls141.wav (deflated 8%)
  adding: content/negtive_old/rc2calcalls311.wav (deflated 6%)
  adding: content/negtive_old/rc2calcalls74.wav (deflated 5%)
  adding: content/negtive_old/ro33cacalls74.wav (deflated 8%)
  adding: content/negtive_old/ro33cacalls295.wav (deflated 8%)
  adding: content/negtive_old/rc2calcalls180.wav (deflated 9%)
  adding: content/negtive_old/ro33cacalls196.wav (deflated 7%)
  adding: content/negtive_old/ro33cacalls419.wav (deflated 4%)
  adding: content/negtive_old/ro33cacalls219.wav (deflated 7%)
  adding: content/negtive_old/ro33cacalls414.wav (deflated 7%)
  adding: content/negtive_old/rc2calcalls72.wav (deflated 5%)
  adding: content/negtive_old/ro33cacalls345.wav (deflated 7%)
  adding: conten

In [ ]:
!zip -r "/content/calls_MAG_SAME_grayscaless.zip" "/content/plot_calls"

  adding: content/plot_calls/ (stored 0%)
  adding: content/plot_calls/round3_callscalls227.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls218.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls264.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls335.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls42.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls136.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls325.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls262.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls103.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls164.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls357.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls73.png (deflated 2%)
  adding: content/plot_calls/round3_callscalls48.png (deflated 2%)
  adding: content/plot_calls/rnd2_clcalls137.png (deflated 2%)
  adding: content/plot_calls/round3_cal

In [ ]:
!zip -r "/content/test_SAME_MAG.zip" "/content/test/"

  adding: content/test/ (stored 0%)
  adding: content/test/.ipynb_checkpoints/ (stored 0%)
  adding: content/test/test_calls/ (stored 0%)
  adding: content/test/test_calls/testcalls32.png (deflated 2%)
  adding: content/test/test_calls/testcalls39.png (deflated 2%)
  adding: content/test/test_calls/testcalls79.png (deflated 2%)
  adding: content/test/test_calls/testcalls93.png (deflated 2%)
  adding: content/test/test_calls/testcalls64.png (deflated 2%)
  adding: content/test/test_calls/testcalls33.png (deflated 2%)
  adding: content/test/test_calls/testcalls57.png (deflated 2%)
  adding: content/test/test_calls/testcalls15.png (deflated 2%)
  adding: content/test/test_calls/testcalls42.png (deflated 2%)
  adding: content/test/test_calls/testcalls61.png (deflated 2%)
  adding: content/test/test_calls/testcalls62.png (deflated 2%)
  adding: content/test/test_calls/testcalls87.png (deflated 2%)
  adding: content/test/test_calls/testcalls101.png (deflated 2%)
  adding: content/test/test_c

In [ ]:
!pwd

/content/OrcasoundLab09272017_Test/wav


In [ ]:
!unzip "/content/No_plot_MAG2.zip"

Archive:  /content/No_plot_MAG2.zip
   creating: content/no_calls_plot2/
  inflating: content/no_calls_plot2/rc2calcalls164.png  
  inflating: content/no_calls_plot2/rc2calcalls92.png  
  inflating: content/no_calls_plot2/rc2calcalls99.png  
  inflating: content/no_calls_plot2/rc2calcalls224.png  
  inflating: content/no_calls_plot2/rc2calcalls56.png  
  inflating: content/no_calls_plot2/rc2calcalls216.png  
  inflating: content/no_calls_plot2/ro33cacalls57.png  
  inflating: content/no_calls_plot2/ro33cacalls115.png  
  inflating: content/no_calls_plot2/ro33cacalls188.png  
  inflating: content/no_calls_plot2/ro33cacalls248.png  
  inflating: content/no_calls_plot2/ro33cacalls411.png  
  inflating: content/no_calls_plot2/rc2calcalls155.png  
  inflating: content/no_calls_plot2/rc2calcalls187.png  
  inflating: content/no_calls_plot2/ro33cacalls349.png  
  inflating: content/no_calls_plot2/ro33cacalls305.png  
  inflating: content/no_calls_plot2/ro33cacalls243.png  
  inflating: conten

In [ ]:
import matplotlib
matplotlib.use("Agg")



from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications import ResNet152V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
from keras.applications import InceptionResNetV2
import sys, numpy
import os
from keras import optimizers, regularizers
from keras.layers import Dropout, Flatten, Dense, Activation, Input
from keras import callbacks, regularizers
from keras.models import load_model
import matplotlib.pyplot as plt 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.engine import  Model
from keras.layers import Conv2D, GlobalAveragePooling2D
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
from keras.applications import InceptionResNetV2
import sys, numpy
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.models import load_model
import matplotlib.pyplot as plt 
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.engine import  Model
from keras.layers import Conv2D, GlobalAveragePooling2D
from sklearn import metrics
import itertools
import tensorflow as tf

from google.colab import files
import datetime, os
%matplotlib inline

In [ ]:
import os

train_data_path = '/content/train/'
test_data_path = '/content/test/'

#Parametres
nb_testn_samples = sum(len(files) for _, _, files in os.walk("/content/test"))
print(nb_testn_samples)
nb_train_samples = sum(len(files) for _, _, files in os.walk("/content/train"))

print(nb_train_samples)
epochs = 250
batch_size = 41
print(nb_train_samples)


202
1572
1572


In [ ]:
import cv2
size=cv2.imread("/content/test/calls/testcalls10.png")
size.shape

(288, 432, 3)

In [ ]:
img_width, img_height=288,432

In [ ]:
train

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


model = Sequential()
model.add(Conv2D(64, (5, 5), padding = 'same', strides=3, input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv2D(64, (5, 5), strides=3, padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.6))



model.add(Conv2D(128, (5, 5), strides=3, padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.6))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=3e-5),
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 96, 144, 64)       4864      
_________________________________________________________________
activation_13 (Activation)   (None, 96, 144, 64)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 96, 144, 64)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 48, 64)        102464    
_________________________________________________________________
activation_14 (Activation)   (None, 32, 48, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 32, 48, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 16, 128)      

In [ ]:

train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_batchsize = 157
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=train_batchsize,
    class_mode='binary',
shuffle=True)



history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // 157,
   epochs=280
    )

model.save('srkw_v2_new_gray2.h5')

Found 1572 images belonging to 3 classes.
Epoch 1/280
10/10 [==============================] - 80s 8s/step - loss: -1.2592 - accuracy: 0.4622
Epoch 2/280
10/10 [==============================] - 79s 8s/step - loss: -7.5500 - accuracy: 0.4996
Epoch 3/280
 2/10 [=====>........................] - ETA: 1:20 - loss: -12.6483 - accuracy: 0.5064